In [745]:
import re
from collections import Counter
import numpy as np

In [746]:
with open('input.txt') as f:
    input = f.readline().strip('\n')

input

'020D78804D397973DB5B934D9280CC9F43080286957D9F60923592619D3230047C0109763976295356007365B37539ADE687F333EA8469200B666F5DC84E80232FC2C91B8490041332EB4006C4759775933530052C0119FAA7CB6ED57B9BBFBDC153004B0024299B490E537AFE3DA069EC507800370980F96F924A4F1E0495F691259198031C95AEF587B85B254F49C27AA2640082490F4B0F9802B2CFDA0094D5FB5D626E32B16D300565398DC6AFF600A080371BA12C1900042A37C398490F67BDDB131802928F5A009080351DA1FC441006A3C46C82020084FC1BE07CEA298029A008CCF08E5ED4689FD73BAA4510C009981C20056E2E4FAACA36000A10600D45A8750CC8010989716A299002171E634439200B47001009C749C7591BD7D0431002A4A73029866200F1277D7D8570043123A976AD72FFBD9CC80501A00AE677F5A43D8DB54D5FDECB7C8DEB0C77F8683005FC0109FCE7C89252E72693370545007A29C5B832E017CFF3E6B262126E7298FA1CC4A072E0054F5FBECC06671FE7D2C802359B56A0040245924585400F40313580B9B10031C00A500354009100300081D50028C00C1002C005BA300204008200FB50033F70028001FE60053A7E93957E1D09940209B7195A56BCC75AE7F18D46E273882402CCD006A600084C1D8ED0E8401D8A90BE12CCF2F4C4ADA602013BC40

In [747]:
def char_generator(input):
    result = ''
    i = 0

    while (i < len(input)):

        yield input[i]
        i += 1

In [748]:
def from_hexa(hexa_string):
    hexa_key = {
        '0' : '0000',
        '1' : '0001',
        '2' : '0010',
        '3' : '0011',
        '4' : '0100',
        '5' : '0101',
        '6' : '0110',
        '7' : '0111',
        '8' : '1000',
        '9' : '1001',
        'A' : '1010',
        'B' : '1011',
        'C' : '1100',
        'D' : '1101',
        'E' : '1110',
        'F' : '1111'
    }

    return ''.join([hexa_key[c] for c in hexa_string])

In [749]:
def get_batch_int(input_gen, amount):
    result = ''

    try:
        for _ in range(amount):
            result += next(input_gen)
    except StopIteration:
        pass

    return int(result, 2)

In [750]:
def get_batch_bin(input_gen, amount):
    result = ''

    try:
        for _ in range(amount):
            result += next(input_gen)
    except StopIteration:
        print(f'Chyba v batch_bin: {result=}')
        pass
    
    #print(f'batch_bin: {result=} {amount=}')
    return result

In [751]:
def get_literal(input_gen):
    literal = ''
    flag = 1

    while True:
        new_packet = get_batch_bin(input_gen, 5)

        flag = new_packet[0]
        literal += new_packet[1:]
        print(f'{new_packet=} {flag=} {literal=}')

        if int(flag) == 0:
            break

    
    #print(literal)
    return int(literal, 2)

In [752]:
def decode(input_gen, packet_limit, level):
    print(f'{level}LVL')

    
    if (packet_limit - 3) < 0:
        get_batch_int(input_gen, packet_limit)
        return 0, 0
    version = get_batch_int(input_gen, 3)
    packet_limit -= 3
    version_sum = version

    if (packet_limit - 3) < 0:
        get_batch_int(input_gen, packet_limit)
        return 0, 0
    type = get_batch_int(input_gen, 3)
    packet_limit -= 3
    version_length = 6

    #literal
    if type == 4:
        literal = get_literal(input_gen)
        print(f'{version=} {type=} {literal=}')
        version_length += 5
    else:
        if (packet_limit - 1) < 0:
            get_batch_int(input_gen, packet_limit)
            return 0, 0
        lenght_type = get_batch_int(input_gen, 1)
        packet_limit -= 1
        version_length += 1

        if lenght_type == 0:
            #15bits total lengths in bits
            if (packet_limit - 15) < 0:
                get_batch_int(input_gen, packet_limit)
                return 0, 0

            subpackets_total_len = get_batch_int(input_gen, 15)
            print(f'{version=} {type=} {lenght_type=} {subpackets_total_len=}')
            version_length += 15

            while True:
                print(f'INNER {subpackets_total_len=}')
                if subpackets_total_len < 11:
                    print(f'Too low subpacket {subpackets_total_len=}')
                    break
                elif subpackets_total_len > 21:
                    sub_sum, sub_len = decode(input_gen, level + 1)
                    
                    
                    subpackets_total_len -= sub_len

                    version_sum += sub_sum
                    version_length += sub_len
                    
                    
                else:
                    if (packet_limit - subpackets_total_len) < 0:
                        get_batch_int(input_gen, packet_limit)
                        return 0, 0

                    subbacket = char_generator(get_batch_bin(input_gen, subpackets_total_len))
                    sub_sum, sub_len = decode(subbacket, level + 1)

                    subpackets_total_len -= sub_len

                    version_sum += sub_sum
                    version_length += sub_len
                    break

                print(f'REST of subpackets: {subpackets_total_len=}')

        elif lenght_type == 1:
            #11bit subpackets coming immediately

            if (packet_limit - 11) < 0:
                get_batch_int(input_gen, packet_limit)
                return 0, 0

            count_coming_subpackets = get_batch_int(input_gen, 11)
            print(f'{version=} {type=} {lenght_type=} {count_coming_subpackets=}')

            version_length += 11

            for _ in range(count_coming_subpackets):
                sub_sum, sub_len = decode(input_gen, level + 1)
                
                version_sum += sub_sum
                version_length += sub_len

    print(f'{level}END {version_sum=} {version_length=}')
    return version_sum, version_length



In [753]:
len('000000000000000001011000010001010110100010111000001000000000101111000110000010001101000000')

90

In [755]:
                                                                                                                                                                                                                                                                                    OP 0 'VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB'
                                                                                                                                                                                                                                                                            OP 1 'VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC' 
     '000000100000110101 111000100000000100 11010011100101111001011100111101101101011 0111001001101001 101100100101000000011 001100100111110100001 1000010000000001010000 1101001010101111 1011001111101100 0001001001000110 1011001001001100 0011001110100110 Z0100011000000000100011111 00000000010000100101110110001110010111011000 101001010100110101011000000000011100110110010110110011011101010011100110101101111001101000011111110011001100111110101010000100011010010010000000001011011001100110111101011101110010000100111010000000001000110010111111000010110010010001101110000100100100000000010000010011001100101110101101000000000001101100010001110101100101110111010110010011001101010011000000000101001011000000000100011001111110101010011111001011011011101101010101111011100110111011111110111101110000010101001100000000010010110000000000100100001010011001101101001001000011100101001101111010111111100011110110100000011010011110110001010000011110000000000000110111000010011000000011111001011011111001001001001010010011110001111000000100100101011111011010010001001001011001000110011000000000110001110010010101101011101111010110000111101110000101101100100101010011110100100111000010011110101010001001100100000000001000001001001001000011110100101100001111100110000000001010110010110011111101101000000000100101001101010111111011010111010110001001101110001100101011000101101101001100000000010101100101001110011000110111000110101011111111011000000000101000001000000000110111000110111010000100101100000110010000000000000100001010100011011111000011100110000100100100001111011001111011110111011011000100110001100000000010100100101000111101011010000000001001000010000000001101010001110110100001111111000100010000010000000001101010001111000100011011001000001000000010000000001000010011111100000110111110000001111100111010100010100110000000001010011010000000001000110011001111000010001110010111101101010001101000100111111101011100111011101010100100010100010000110000000000100110011000000111000010000000000101011011100010111001001111101010101100101000110110000000000000101000010000011000000000110101000101101010000111010100001100110010000000000100001001100010010111000101101010001010011001000000000010000101110001111001100011010001000011100100100000000010110100011100000000000100000000100111000111010010011100011101011001000110111101011111010000010000110001000000000010101001001010011100110000001010011000011001100010000000001111000100100111011111010111110110000101011100000000010000110001001000111010100101110110101011010111001011111111101111011001110011001000000001010000000110100000000010101110011001110111111101011010010000111101100011011011010101001101010111111101111011001011011111001000110111101011000011000111011111111000011010000011000000000101111111000000000100001001111111001110011111001000100100100101001011100111001001101001001100110111000001010100010100000000011110100010100111000101101110000011001011100000000101111100111111110011111001101011001001100010000100100110111001110010100110001111101000011100110001001010000001110010111000000000010101001111010111111011111011001100000001100110011100011111111001111101001011001000000000100011010110011011010101101010000000000100000000100100010110010010010001011000010101000000000011110100000000110001001101011000000010111001101100010000000000110001110000000000101001010000000000110101010000000000100100010000000000110000000000001000000111010101000000000010100011000000000011000001000000000010110000000000010110111010001100000000001000000100000000001000001000000000111110110101000000000011001111110111000000000010100000000000000111111110011000000000010100111010011111101001001110010101011111100001110100001001100101000000001000001001101101110001100101011010010101101011110011000111010110101110011111110001100011010100011011100010011100111000100000100100000000101100110011010000000001101010011000000000000010000100110000011101100011101101000011101000010000000001110110001010100100001011111000010010110011001111001011110100110001001010110110100110000000100000000100111011110001000000000110111000110000010001001101100000100010000000000000100001101100010011011000011110010001010001000100000000011101100000100111000111101110001100101010000000000000101101110000110010001000000000111100111010110000000001011010011000111011100010111111111000101000101100100101011011010000101111001011011011101011101011010010001010010000000001101111000101100000000010011100110111111110000100011000001100010100110111111110000100011000001100111100111101100101010011000100000000011000100101110001011010011111010111011001101110100111001011011110100011100111011000100000100000000010000010111100101001101010011100100111000010010001001101011001010010100111000101100010011111100011000110100100111110101001011100110100000010000110000000010000110011100000100001001001100100100100101010110101110011001010010010010000000010001011001110000000000110101011100010000000001101110000100100100001000111000111010000000000000001000010011000011011010001011110100011010101000111111100100000000011001010001101000000000101001001111011011100010011110110011110110111000100111001100110111010111011110010101001100100000000100100101101111000000000101000001000000000111111000111011110000011000100011100110001000000000110001000101011110000110110100001100101001100111101110100001111001100000000010100111001001101010110100101101011111011101101001010010011110011000011001001101011000100001101010000000001001110011001100010101110110011111011110000'
                                                                                                                                                                                                                                                                                    LIT  'VVVTTTAAAAABBBBBCCCCC'
                                                                                                                                                                                                                                                                                          1234567890123456789012345678901234567890

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
len(input)

1350

In [ ]:
input

'020D78804D397973DB5B934D9280CC9F43080286957D9F60923592619D3230047C0109763976295356007365B37539ADE687F333EA8469200B666F5DC84E80232FC2C91B8490041332EB4006C4759775933530052C0119FAA7CB6ED57B9BBFBDC153004B0024299B490E537AFE3DA069EC507800370980F96F924A4F1E0495F691259198031C95AEF587B85B254F49C27AA2640082490F4B0F9802B2CFDA0094D5FB5D626E32B16D300565398DC6AFF600A080371BA12C1900042A37C398490F67BDDB131802928F5A009080351DA1FC441006A3C46C82020084FC1BE07CEA298029A008CCF08E5ED4689FD73BAA4510C009981C20056E2E4FAACA36000A10600D45A8750CC8010989716A299002171E634439200B47001009C749C7591BD7D0431002A4A73029866200F1277D7D8570043123A976AD72FFBD9CC80501A00AE677F5A43D8DB54D5FDECB7C8DEB0C77F8683005FC0109FCE7C89252E72693370545007A29C5B832E017CFF3E6B262126E7298FA1CC4A072E0054F5FBECC06671FE7D2C802359B56A0040245924585400F40313580B9B10031C00A500354009100300081D50028C00C1002C005BA300204008200FB50033F70028001FE60053A7E93957E1D09940209B7195A56BCC75AE7F18D46E273882402CCD006A600084C1D8ED0E8401D8A90BE12CCF2F4C4ADA602013BC40

In [ ]:
len(test)

5400

In [ ]:
'000000100000110101111000100000000100110100111001011110010111001111011011010110111001001101001101100100101000000011001100100111110100001100001000000000101000011010010101011111011001111101100000100100100011010110010010011000011001110100110010001100000000010001111100000000010000100101110110001110010111011000101001010100110101011000000000011100110110010110110011011101010011100110101101111001101000011111110011001100111110101010000100011010010010000000001011011001100110111101011101110010000100111010000000001000110010111111000010110010010001101110000100100100000000010000010011001100101110101101000000000001101100010001110101100101110111010110010011001101010011000000000101001011000000000100011001111110101010011111001011011011101101010101111011100110111011111110111101110000010101001100000000010010110000000000100100001010011001101101001001000011100101001101111010111111100011110110100000011010011110110001010000011110000000000000110111000010011000000011111001011011111001001001001010010011110001111000000100100101011111011010010001001001011001000110011000000000110001110010010101101011101111010110000111101110000101101100100101010011110100100111000010011110101010001001100100000000001000001001001001000011110100101100001111100110000000001010110010110011111101101000000000100101001101010111111011010111010110001001101110001100101011000101101101001100000000010101100101001110011000110111000110101011111111011000000000101000001000000000110111000110111010000100101100000110010000000000000100001010100011011111000011100110000100100100001111011001111011110111011011000100110001100000000010100100101000111101011010000000001001000010000000001101010001110110100001111111000100010000010000000001101010001111000100011011001000001000000010000000001000010011111100000110111110000001111100111010100010100110000000001010011010000000001000110011001111000010001110010111101101010001101000100111111101011100111011101010100100010100010000110000000000100110011000000111000010000000000101011011100010111001001111101010101100101000110110000000000000101000010000011000000000110101000101101010000111010100001100110010000000000100001001100010010111000101101010001010011001000000000010000101110001111001100011010001000011100100100000000010110100011100000000000100000000100111000111010010011100011101011001000110111101011111010000010000110001000000000010101001001010011100110000001010011000011001100010000000001111000100100111011111010111110110000101011100000000010000110001001000111010100101110110101011010111001011111111101111011001110011001000000001010000000110100000000010101110011001110111111101011010010000111101100011011011010101001101010111111101111011001011011111001000110111101011000011000111011111111000011010000011000000000101111111000000000100001001111111001110011111001000100100100101001011100111001001101001001100110111000001010100010100000000011110100010100111000101101110000011001011100000000101111100111111110011111001101011001001100010000100100110111001110010100110001111101000011100110001001010000001110010111000000000010101001111010111111011111011001100000001100110011100011111111001111101001011001000000000100011010110011011010101101010000000000100000000100100010110010010010001011000010101000000000011110100000000110001001101011000000010111001101100010000000000110001110000000000101001010000000000110101010000000000100100010000000000110000000000001000000111010101000000000010100011000000000011000001000000000010110000000000010110111010001100000000001000000100000000001000001000000000111110110101000000000011001111110111000000000010100000000000000111111110011000000000010100111010011111101001001110010101011111100001110100001001100101000000001000001001101101110001100101011010010101101011110011000111010110101110011111110001100011010100011011100010011100111000100000100100000000101100110011010000000001101010011000000000000010000100110000011101100011101101000011101000010000000001110110001010100100001011111000010010110011001111001011110100110001001010110110100110000000100000000100111011110001000000000110111000110000010001001101100000100010000000000000100001101100010011011000011110010001010001000100000000011101100000100111000111101110001100101010000000000000101101110000110010001000000000111100111010110000000001011010011000111011100010111111111000101000101100100101011011010000101111001011011011101011101011010010001010010000000001101111000101100000000010011100110111111110000100011000001100010100110111111110000100011000001100111100111101100101010011000100000000011000100101110001011010011111010111011001101110100111001011011110100011100111011000100000100000000010000010111100101001101010011100100111000010010001001101011001010010100111000101100010011111100011000110100100111110101001011100110100000010000110000000010000110011100000100001001001100100100100101010110101110011001010010010010000000010001011001110000000000110101011100010000000001101110000100100100001000111000111010000000000000001000010011000011011010001011110100011010101000111111100100000000011001010001101000000000101001001111011011100010011110110011110110111000100111001100110111010111011110010101001100100000000100100101101111000000000101000001000000000111111000111011110000011000100011100110001000000000110001000101011110000110110100001100101001100111101110100001111001100000000010100111001001101010110100101101011111011101101001010010011110011000011001001101011000100001101010000000001001110011001100010101110110011111011110000'

In [ ]:
test

'000000100000110101111000100000000100110100111001011110010111001111011011010110111001001101001101100100101000000011001100100111110100001100001000000000101000011010010101011111011001111101100000100100100011010110010010011000011001110100110010001100000000010001111100000000010000100101110110001110010111011000101001010100110101011000000000011100110110010110110011011101010011100110101101111001101000011111110011001100111110101010000100011010010010000000001011011001100110111101011101110010000100111010000000001000110010111111000010110010010001101110000100100100000000010000010011001100101110101101000000000001101100010001110101100101110111010110010011001101010011000000000101001011000000000100011001111110101010011111001011011011101101010101111011100110111011111110111101110000010101001100000000010010110000000000100100001010011001101101001001000011100101001101111010111111100011110110100000011010011110110001010000011110000000000000110111000010011000000011111001011011111001001001001010010011110001111

In [756]:
#test = from_hexa('A0016C880162017C3686B18A3D4780')
test = from_hexa(input)
test_gen = char_generator(test)

decode(test_gen, 1)

TypeError: decode() missing 1 required positional argument: 'level'